## Portfolio Optimization using Quadratic Unconstraint Binary Optimization (QUBOs) with Constraints

In [1]:
import pygrnd
from pygrnd.optimize.sat_ucp import *


In [2]:
import numpy as np
#from functions import Long_Short, Cov_Matrix_Resolution
#from pygrnd.optimize.helpers.qubomatrix import matrix_to_qubo
#from pygrnd.optimize.simulate.anneal import anneal_qubo
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

# Application: Create Market neutral portfolio

 - Pick stocks from a list
 - want to pick 5 long and 5 short positions
 - want minimal variance -> similar to constraint
 
 --> minimizing variance with a QUBO is straightforward
 
 --> Use covariance matrix as Q

### First write matrix in qubo form

A QUBO problem is defined using an upper-diagonal matrix Q, which is an N x N upper-triangular matrix of real weights, and x, a vector of binary variables, as minimizing the function
$$ f(x) = \sum_{i} Q_{i,i} x_{i} + \sum_{i<j} Q_{i,j} x_{i} x_{j}  $$
with $Q_{i,i}$ are the linear coefficients and $Q_{i,j}$ nonzero off-diagonal terms are the quadratic coefficients. Or different representation:
$$ min_{x \in \{0,1\}^{N}} x^{T} Q x $$

## Convert entries $x_i$ to be non-binary

 $\rightarrow$ $\quad$ $-1$ for short position

 $\rightarrow$ $\quad$ $0$ not selected for portfolio

 $\rightarrow$ $\quad$ $+1$ long position
 
 for example: (−1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 which is CBK long, DBK short
 
 PROBLEM!!  x needs to be 0 or 1
 
 SOLUTION: double the size to 42 components
 --> size of $x$ is doubled, first part long positions (21x 0 or 1), second part short positions (21x 0 or 1)
 
 Capture variance and covariance among long and shorts separatly:
 
  $Q_u = \begin{pmatrix}
C & 0 \\
0 & C
\end{pmatrix}$

 where C is the covariance matrix.
 
 Covariance between a long asset i and short in asset j is given by  $-Cov(i,j)$
 
 $Q_u = \begin{pmatrix}
C & -C \\
-C & C
\end{pmatrix}$

for the full unconstrained Qubo matrix $Q_u$



## Adding constraints

 1. exactly 5 long and 5 short positions: $Q_{c1}$
 
 $\sum_i x_i = N$, with $N$ total assets picked
 
 $(\sum_i x_i - N)^2 = 0$
 
 $\sum_i x_i \sum_j x_j - 2 N \sum_i x_i + N = 0$
 
 $\Leftrightarrow minimize \sum_i x_i \sum_j x_j - 2 N \sum_i x_i$
 
 
 $\rightarrow$  $Q_{c1} = \begin{pmatrix}
-2N-1 & 1 & ... \\
1 & -2N-1 & ... \\
...
\end{pmatrix}$
 
 
 
 2. no simultaneous long and short position in the same stock $Q_{c2}$
 
 $Q_{c2} = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}$
 
 
 --> $Q = Q_u + Q_{c1} + Q_{c2}$

In [3]:
bigq=np.zeros((30,30))
cov = np.load("DAXCovMat.npy")
c1=np.zeros((30,30))
qc1=np.zeros((30,30))

print("Cov size:",len(cov)," x ",len(cov))

Cov size: 30  x  30


In [4]:
## function to build the QUBO matrix using the covariance matrix

def MakeQubo(m):
        for x in range(np.size(m,0)):
            for y in range(np.size(m,1)):
                if x>y:
                    m[x,y]=0
                if x<y:
                    m[x,y]=2*m[x,y]
        return(m)

In [5]:
## adding the constraints 1 and 2

def BuildConstraintMatrix1(la,lp,sp):
    big=np.zeros((30,30))
    sa=30-la
    # constraint for the longs
    for x in range(30):
        for y in range(30):
            qc1[x,y]=0
    for x in range(la):
        for y in range(la):
            qc1[x,y]=1
            if x==y:
                qc1[x,y]=-2*lp+1
    # constraint for the shorts
    for x in range(sa):
        for y in range(sa):
            qc1[la+x,la+y]=1
            if x==y:
                qc1[la+x,la+y]=-2*sp+1
    for x in range(la):
        for y in range(la):
            c1[x,y]=cov[x,y]
    # cov for short-short
    for x in range(sa):
        for y in range(sa):
            c1[la+x,la+y]=cov[la+x,la+y]
    # cov for long-short
    for x in range(sa):
        for y in range(la):
            c1[la+x,y] = -cov[la+x,y]
    for x in range(la):
        for y in range(sa):
            c1[x,la+y] = -cov[x,la+y]
    plt.matshow(c1)
    plt.matshow(qc1)
    big=MakeQubo(c1+qc1)
    for x in range(30):
        for y in range(30):
            bigq[x,y]=big[x,y]
    return()

# here we try the D-Wave greedy solver:

In [6]:
La = widgets.IntSlider(
    min=0,
    max=30,
    step=1,
    description='LongvShort:',
    value=10
)
Long = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Buy:',
    value=5
)
Short = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Sell:',
    value=5
)

ui = widgets.HBox([widgets.HBox([La, Long, Short])])

out = widgets.interactive_output(BuildConstraintMatrix1, {'la': La, 'lp': Long, 'sp': Short})

outputCovPlot = widgets.Output()

button = widgets.Button(description="SOLVE!")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        #qubo = matrix_to_qubo(bigq)
        #result=anneal_qubo(qubo, num_anneals=50)
        #qubo_solution = result.best.state
        #res,vec=MCgradientSearch(bigq,10000)
        res,vec=dwaveGreedySolver(bigq,100)
        #print("Variable assignment:", qubo_solution)
        #print("value:", result.best.value)
        #print("Constraints satisfied?", qubo.is_solution_valid(qubo_solution))
        #for key in qubo_solution.keys():
            #if qubo_solution[key] == 1:
                #if key < La.value:
                    #print("buy:", key)
                #else:
                    #print("sell:", key)
        for i in range(len(vec)):
            if vec[i] == 1:
                if i <La.value:
                    print("buy: ",i)
                else:
                    print("sell: ",i)

button.on_click(on_button_clicked)

tab = widgets.Tab([outputCovPlot, out, output])
tab.set_title(0, 'Input')
tab.set_title(1, 'Heatmap')
tab.set_title(2, 'Solution')

with outputCovPlot:
    plt.figure(figsize=(20,20))
    sns.heatmap(cov, cmap='coolwarm', annot=True, linewidth=.3)
    plt.show()

display(ui, button, tab)

Button(description='SOLVE!', style=ButtonStyle())